In [1]:
import numpy as np
import tensorflow as tf
import datetime

In [2]:
# roadmap:
# 1-implement dcgan in tf and observe generated examples
# 2-investigate tf dcgan implementation

# 3-implement dcgan in torch and observed generated examples
# 4-investigate torch dcgan implementation

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# [0,255] -> [0,1] -> [-1,1]
x_train = (x_train/255.) * 2. - 1.

x_train = np.expand_dims(x_train,axis=3)
x_train = tf.cast(x_train,dtype=tf.float32)

train_ds = tf.data.Dataset.from_tensor_slices(x_train)
train_ds = train_ds.shuffle(1000).batch(64)
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)

2022-04-30 23:32:33.714503: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# GENERATOR
gen_inputs = tf.keras.Input(shape=(100,))
x = tf.keras.layers.Dense(units=(7 * 7 * 128),activation='relu')(gen_inputs)
x = tf.keras.layers.Reshape(target_shape=(7,7,128))(x)
x = tf.keras.layers.Conv2DTranspose(filters=128,kernel_size=5,strides=1,activation='relu',padding='same')(x)
x = tf.keras.layers.Conv2DTranspose(filters=64,kernel_size=5,strides=2,activation='relu',padding='same')(x)
outputs = tf.keras.layers.Conv2DTranspose(filters=1,kernel_size=5,strides=2,activation='tanh',padding='same')(x)
Generator = tf.keras.Model(inputs=gen_inputs, outputs=[outputs])

# DISCRIMINATOR
disc_inputs = tf.keras.Input(shape=(28,28,1))
y = tf.keras.layers.Conv2D(filters=64,kernel_size=5,strides=2,padding='same')(disc_inputs)
y = tf.keras.layers.LeakyReLU(alpha=0.2)(y)
y = tf.keras.layers.Conv2D(filters=128,kernel_size=5,strides=2,padding='same')(y)
y = tf.keras.layers.LeakyReLU(alpha=0.2)(y)
y = tf.keras.layers.Flatten()(y)
outputs = tf.keras.layers.Dense(units=1)(y)
Discriminator = tf.keras.Model(inputs=disc_inputs, outputs=[outputs])


In [5]:
# loss function
loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True,label_smoothing=0.0)

# optimizers
gen_optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0002)
disc_optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0002)

# metrics
disc_loss_tracker = tf.keras.metrics.Mean(name='disc_loss')
gen_loss_tracker = tf.keras.metrics.Mean(name='gen_loss')

# tensorboard
experiment_name = 'deneme'
log_dir = '../logs/'+experiment_name
summary_writer = tf.summary.create_file_writer(log_dir)

latent_code_size = 100
# fix latent code to track improvement
latent_code4visualization = tf.random.normal(shape=(16,latent_code_size))
epochs = 50

for epoch in range(epochs):

    for _,real_imgs in train_ds.enumerate():
        
        # PART 1: DISC TRAINING, fixed generator
        latent_code = tf.random.normal(shape=(real_imgs.shape[0],latent_code_size))

        with tf.GradientTape() as disc_tape:
            # generate fake images
            generated_imgs = Generator(latent_code)

            # forward pass real and fake images
            real_preds,fake_preds = Discriminator(real_imgs),Discriminator(generated_imgs)
            y_pred = tf.concat([real_preds,fake_preds],axis=0)
            y_true = tf.concat([tf.ones_like(real_preds),tf.zeros_like(fake_preds)],axis=0)
            # adding random noise to labels
            y_true += 0.05 * tf.random.uniform(tf.shape(y_true))
            
            # compute loss
            disc_loss = loss_fn(y_true=y_true,y_pred=y_pred)

        # compute disc gradients
        disc_gradients = disc_tape.gradient(disc_loss,Discriminator.variables)

        # update disc weights
        disc_optimizer.apply_gradients(zip(disc_gradients, Discriminator.variables))

        # update disc metrics
        disc_loss_tracker.update_state(disc_loss)


        # PART 2: GEN TRAINING, fixed discriminator
        latent_code = tf.random.normal(shape=(real_imgs.shape[0],latent_code_size))

        with tf.GradientTape() as gen_tape:
            # generate fake images
            generated_imgs = Generator(latent_code)

            # forward pass only images
            fake_preds = Discriminator(generated_imgs)

            # compute loss
            gen_loss = loss_fn(y_true=tf.ones_like(fake_preds),y_pred=fake_preds)

        # compute gen gradients
        gen_gradients = gen_tape.gradient(gen_loss,Generator.variables)

        # update gen weights
        gen_optimizer.apply_gradients(zip(gen_gradients, Generator.variables))

        # update gen metrics
        gen_loss_tracker.update_state(gen_loss)


    # generate sample images per epoch
    test_generated_imgs = Generator(latent_code4visualization)
    test_generated_imgs = (test_generated_imgs+1.)/2.
    
    # display and record metrics at the end of each epoch.
    with summary_writer.as_default():
        tf.summary.scalar('disc_loss', disc_loss_tracker.result(), step=epoch)
        tf.summary.scalar('gen_loss', gen_loss_tracker.result(), step=epoch)
        tf.summary.image(name='test_samples',data=test_generated_imgs,max_outputs=test_generated_imgs.shape[0],step=epoch)

    disc_loss,gen_loss = disc_loss_tracker.result(),gen_loss_tracker.result()
    print(f'epoch: {epoch}, disc_loss: {disc_loss:.4f}, gen_loss: {gen_loss:.4f}')

    # reset metric states
    disc_loss_tracker.reset_state()
    gen_loss_tracker.reset_state()

KeyboardInterrupt: 